# Mapa Folium HeatMap

In [1]:
# heatmap con movimiento
import folium
from folium.plugins import HeatMapWithTime as HMWT
from folium.plugins import HeatMap as HM
import pandas as pd


df=pd.read_csv('mex_earthquakes.csv')

df.head()

,_id,Fecha,Hora,Latitud,Longitud,Profundidad,Intensidad
0,"ObjectId(""5db7805d3771a8f231530f47"")",2019-10-28,09:08:45,26.04,-107.89,10.0 km,4.0
1,"ObjectId(""5db7805d3771a8f231530f49"")",2019-10-28,08:15:02,14.13,-93.34,19.0 km,4.1
2,"ObjectId(""5db7805d3771a8f231530f4b"")",2019-10-28,04:59:59,17.90,-95.08,120.0 km,3.8
3,"ObjectId(""5db7805d3771a8f231530f4d"")",2019-10-28,04:55:15,16.69,-99.42,7.0 km,3.4
4,"ObjectId(""5db7805d3771a8f231530f4f"")",2019-10-28,04:34:54,16.30,-98.25,13.0 km,3.3


In [2]:
df['date_time']=df.Fecha+' '+df.Hora

df.head()

,_id,Fecha,Hora,Latitud,Longitud,Profundidad,Intensidad,date_time
0,"ObjectId(""5db7805d3771a8f231530f47"")",2019-10-28,09:08:45,26.04,-107.89,10.0 km,4.0,2019-10-28 09:08:45
1,"ObjectId(""5db7805d3771a8f231530f49"")",2019-10-28,08:15:02,14.13,-93.34,19.0 km,4.1,2019-10-28 08:15:02
2,"ObjectId(""5db7805d3771a8f231530f4b"")",2019-10-28,04:59:59,17.90,-95.08,120.0 km,3.8,2019-10-28 04:59:59
3,"ObjectId(""5db7805d3771a8f231530f4d"")",2019-10-28,04:55:15,16.69,-99.42,7.0 km,3.4,2019-10-28 04:55:15
4,"ObjectId(""5db7805d3771a8f231530f4f"")",2019-10-28,04:34:54,16.30,-98.25,13.0 km,3.3,2019-10-28 04:34:54


In [3]:
# transformar segun datos de tiempo

df.date_time=pd.to_datetime(df.date_time, format='%Y-%m-%d %H:%M:%S')

df['mes']=df.date_time.apply(lambda x: x.month)

df['semana']=df.date_time.apply(lambda x: x.week)

df['dia']=df.date_time.apply(lambda x: x.day)

df['hora']=df.date_time.apply(lambda x: x.hour)

In [4]:
# inicializa el mapa

def mapa_base(default_location=[23.634501, -102.552784], default_zoom_start=4.6):
    
    
    return folium.Map(location=default_location, 
                      control_scale=True, 
                      zoom_start=default_zoom_start)
    

In [5]:
mapa_base()

In [6]:
# se copia df y mes>4

df_copy = df[df.mes>4].copy()
df_copy['count'] = 1

In [7]:
# datos para heatmap

df_copy[['Latitud', 
         'Longitud', 
         'count']].groupby(['Latitud', 
                             'Longitud']).sum().sort_values('count', 
                                                                   ascending=False).head(10)

count
Latitud Longitud       
30.82   -114.22       4
16.26   -98.10        3
15.39   -94.75        2
19.22   -104.98       2
16.23   -98.07        2
16.31   -98.25        2
        -98.22        2
18.84   -98.83        2
16.21   -98.11        2
16.99   -99.50        2

In [8]:
# heatmap

HM(data=df_copy[['Latitud', 
                  'Longitud', 
                  'count']].groupby(['Latitud', 
                                      'Longitud']).sum().reset_index().values.tolist(), 
    radius=8, 
        max_zoom=13).add_to(mapa_base()) # pon los datos en heatmap

#base_map.save('base_map.html')  # guarda

base_map   # representa

In [9]:
df_hour_list = []
for hour in df_copy.semana.sort_values().unique():
    df_hour_list.append(df_copy.loc[df_copy.semana == hour, 
                                    ['Latitud', 
                                     'Longitud', 
                                     'count']].groupby(['Latitud', 
                                                        'Longitud']).sum().reset_index().values.tolist())

In [10]:
len(df_hour_list)

14

In [11]:
mapa=generateBaseMap() # crea mapa

HMWT(df_hour_list, 
     radius=5, 
     gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, 
     min_opacity=0.5, 
     max_opacity=0.8, 
     use_local_extrema=True).add_to(mapa) # datos pata heatmap con tiempo


mapa  # representa

In [12]:
mapa.save('mapa.html') # guarda